In [2]:
import pandas as pd
import numpy as np
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook, curdoc
from bokeh.models import HoverTool

from pynba import constants
from pynba.team_info import team_abb_to_id
from pynba.plot import bokeh_theme


output_notebook()
doc = curdoc()
theme = bokeh_theme("dark")
doc.theme = theme

Loading BokehJS ...

In [34]:
thing = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]}).abs()
thing.max().max()

6

In [46]:
logo_url("UTA", "nba", 2022)

'https://cdn.nba.com/logos/nba/1610612762/primary/L/logo.svg'

In [42]:
def logo_url(team, league, year):
    if league == constants.G_LEAGUE:
        return f"https://stats.gleague.nba.com/media/img/teams/logos/{team}.svg"
    if league == constants.WNBA:
        return f"https://stats.wnba.com/media/img/teams/logos/{team}.svg"
    if league == constants.NBA:
        team_id = team_abb_to_id(league, year)[team]
        return f"https://cdn.nba.com/logos/nba/{team_id}/primary/L/logo.svg"
    raise Exception(f"Invalid league {league} provided")


def plot_logos(data, league, year, size, fig):
    source = ColumnDataSource(data)
    source.data["logo_url"] = [logo_url(team, league, year) for team in data.index]
    source.data["team"] = data.index
    
    images = fig.image_url(url="logo_url", x=data.columns[0], y=data.columns[1], source=source, w=size, h=size, w_units="screen", h_units="screen", anchor="center")
    circles = fig.circle(x=data.columns[0], y=data.columns[1], size=size, alpha=0, source=source)
    
    bound = data.abs().max().max()
    fig.circle(x=[bound, -bound], y=[bound, -bound], size=size, alpha=0)
    
    tooltips = [("Team", "@team"), (data.columns[0], f"@{{{data.columns[0]}}}{{0.0}}"), (data.columns[1], f"@{{{data.columns[1]}}}{{0.0}}")]
    hover_tool = HoverTool(tooltips=tooltips, renderers=[images, circles])
    fig.add_tools(hover_tool)


In [45]:
league = constants.NBA
year = 2022
season_type = constants.REGULAR_SEASON

teams = [
    'TOR',
    'DEN',
    'DET',
    'NOP',
    'MIL',
    'GSW',
    'SAS',
    'POR',
    'LAC',
    'DAL',
    'MEM',
    'PHI',
    'BOS',
    'MIN',
    'CHA',
    'IND',
    'ORL',
    'MIA',
    'HOU',
    'SAC',
    'UTA',
    'BKN',
    'ATL',
    'PHX',
    'WAS',
    'NYK',
    'OKC',
    'LAL',
    'CLE',
    'CHI',
]

off_pm, def_pm = np.random.normal(size=(2, 30)) * 2

data = pd.DataFrame({"Off": off_pm, "Def": def_pm}, index=teams)

fig = figure(title=f"{league} {year} {season_type}", width=1000, height=1000)

plot_logos(data, league, year, size=50, fig=fig)

fig.xaxis.axis_label = "Offensive Rating (pts/poss)"
fig.yaxis.axis_label = "Defensive Rating (pts/poss)"

show(fig)